compound family generator

In [22]:
import re
import pandas as pd
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

df = pd.read_csv ("/home/bhikkhu/Bodhirasa/Dropbox/dpd/csvs/dpd-full.csv", sep="\t", dtype=str)
df = df.fillna("") 
df_length = df.shape[0]

extract compound family names

In [23]:
print("~"*40)
print("extracting compound family names")

test1 = df["Family2"] != ""
test2 = df["Meaning IN CONTEXT"] != ""
test3 = df["Pāli Root"] == ""
test4 = df["Metadata"] != "yes"
filter = test1 & test2 & test3 & test4
compound_family_df = df.loc[filter, ["Family2"]]

#make compound family list
compound_family_list = []
compound_family_df["Family2"].str.split()
compound_family_list = compound_family_df["Family2"].to_list()

#flatten list
compound_family_list = [item for sublist in compound_family_list for item in sublist.split(" ")]
with open ("compound_family_list.csv", "w") as write_list:
	for item in compound_family_list:
		write_list.write(item + "\n")

# import compound_family_list as dataframe
compound_family_list = pd.DataFrame(compound_family_list)

#drop dupes and sort
compound_family_list.drop_duplicates(keep="first", inplace=True)
compound_family_list.sort_values([0], ascending=True, inplace=True)
compound_family_list.reset_index(drop=True, inplace=True)

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
extracting compound family names


generate compound family lists

In [24]:
print("~"*40)
print("generating compound family list")
print("~"*40)

df['Pāli3'] = df['Pāli1'].str.replace(" \d*", "")

test1 = df["Family2"] != ""
test2 = df["Meaning IN CONTEXT"] != ""
test3 = df["Pāli Root"] == ""
test4 = df["Metadata"] != "yes"
filter = test1 & test2 & test3 & test4
df_reduced = df[filter]

df_length = df_reduced.shape[0]


# row count
compound_family_count = compound_family_list.shape[0]

for row in range (compound_family_count): #compound_family_count
    compound_family =  compound_family_list.iloc[row,0]
    
    test1 = df_reduced["Family2"].str.contains(rf"\b{compound_family}\b")
    test2 = df_reduced["Pāli3"] != compound_family
    filter = test1 & test2
    df_filtered = df_reduced.loc[filter, ["Pāli1", "POS", "Meaning IN CONTEXT"]]

    if row % 500 == 0:
        print(f"{row}/{compound_family_count}\t{compound_family}")

    if df_filtered.shape[0] > 0:

     # html conversion

        html = ""
        length = df_filtered.shape[0]
        
        html += f"""<tbody>"""

        for row in range(length):
            cf_pali = df_filtered.iloc[row, 0]
            cf_pos = df_filtered.iloc[row, 1]
            cf_english = df_filtered.iloc[row, 2]
        
            html += f"<tr><th>{cf_pali}</th>"
            html += f"<td>{cf_pos}</td>"
            html += f"<td>{cf_english}</td></tr>"
        html += f"</tbody>"

        with open (f"output/{compound_family}.html", 'w', encoding= "'utf-8") as html_file:
            html_file.write(html)


~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
generating compound family list
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
0/4008	a
500/4008	bhikkhunī
1000/4008	ha


KeyboardInterrupt: 